# Fast Inference Test: Llama-3.2-1B

Tests `fast_inference=True` with vLLM backend on Llama-3.2-1B-Instruct.

**Important:** This notebook includes a kernel shutdown cell at the end.
vLLM does not release GPU memory in single-process mode (Jupyter), so kernel
restart is required between different model tests.

In [1]:
# Environment Setup
from dotenv import load_dotenv
import os
load_dotenv()
print(f"HF_TOKEN loaded: {'Yes' if os.environ.get('HF_TOKEN') else 'No'}")

# CRITICAL: Import unsloth FIRST for proper TRL patching
import unsloth
from unsloth import FastLanguageModel
import transformers
import vllm
import trl
import torch

print(f"unsloth: {unsloth.__version__}")
print(f"transformers: {transformers.__version__}")
print(f"vLLM: {vllm.__version__}")
print(f"TRL: {trl.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

HF_TOKEN loaded: Yes🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/trl/__init__.py:203: UserWarning: TRL currently supports vLLM versions: 0.10.2, 0.11.0, 0.11.1, 0.11.2. You have version 0.14.0rc1.dev201+gadcf682fc.cu130 installed. We recommend installing a supported version to avoid compatibility issues.
  if is_vllm_available():

🦥 Unsloth Zoo will now patch everything to make training faster!unsloth: 2025.12.10
transformers: 5.0.0rc1
vLLM: 0.14.0rc1.dev201+gadcf682fc
TRL: 0.26.2
PyTorch: 2.9.1+cu130
CUDA: True
GPU: NVIDIA GeForce RTX 4080 SUPER

In [2]:
# Test fast_inference=True with Llama-3.2-1B-Instruct
print("=== Llama-3.2-1B Fast Inference Test (vLLM Backend) ===")

from unsloth import FastLanguageModel
from vllm import SamplingParams
import torch
import time

model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length=512,
    load_in_4bit=True,
    fast_inference=True,
    gpu_memory_utilization=0.5,
)
print("✓ Model loaded with fast_inference=True")

FastLanguageModel.for_inference(model)
messages = [{"role": "user", "content": "Say hello in one word."}]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

sampling_params = SamplingParams(temperature=0.1, max_tokens=10)

start = time.time()
outputs = model.fast_generate([prompt], sampling_params=sampling_params)
elapsed = time.time() - start

response = outputs[0].outputs[0].text
print(f"✓ vLLM generation completed in {elapsed:.2f}s")
print(f"  Response: {response}")
print("✓ Llama-3.2-1B fast_inference test PASSED")

=== Llama-3.2-1B Fast Inference Test (vLLM Backend) ===WARNING 01-02 23:09:13 [vllm.py:1427] Current vLLM config is not set.INFO 01-02 23:09:13 [scheduler.py:231] Chunked prefill is enabled with max_num_batched_tokens=2048.INFO 01-02 23:09:13 [vllm.py:609] Disabling NCCL for DP synchronization when using async scheduling.INFO 01-02 23:09:13 [vllm.py:614] Asynchronous scheduling is enabled.INFO 01-02 23:09:13 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture==((====))==  Unsloth 2025.12.10: Fast Llama patching. Transformers: 5.0.0.1. vLLM: 0.14.0rc1.dev201+gadcf682fc.cu130.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu130. CUDA: 8.9. CUDA Toolkit: 13.0. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!Unsloth: v

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
  return self.serializer.to_python(

WARNING 01-02 23:09:20 [arg_utils.py:1196] The global random seed is set to 0. Since VLLM_ENABLE_V1_MULTIPROCESSING is set to False, this may affect the random state of the Python process that launched vLLM.INFO 01-02 23:09:22 [model.py:517] Resolved architecture: LlamaForCausalLMINFO 01-02 23:09:22 [model.py:1688] Using max model len 512WARNING 01-02 23:09:22 [vllm.py:1427] Current vLLM config is not set.INFO 01-02 23:09:22 [scheduler.py:231] Chunked prefill is enabled with max_num_batched_tokens=2048.INFO 01-02 23:09:22 [scheduler.py:231] Chunked prefill is enabled with max_num_batched_tokens=4096.Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'bfloat16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_

/opt/pixi/.pixi/envs/default/lib/python3.13/site-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
  return self.serializer.to_python(

INFO 01-02 23:09:25 [topk_topp_sampler.py:47] Using FlashInfer for top-p & top-k sampling.INFO 01-02 23:09:25 [gpu_model_runner.py:3762] Starting to load model unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit...INFO 01-02 23:09:26 [cuda.py:315] Using AttentionBackendEnum.FLASHINFER backend.INFO 01-02 23:09:26 [bitsandbytes_loader.py:790] Loading weights with BitsAndBytes quantization. May take a while ...INFO 01-02 23:09:27 [weight_utils.py:550] No model.safetensors.index.json found in remote.

...

Unsloth: Will load unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit as a legacy tokenizer.

✓ Model loaded with fast_inference=True

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

✓ vLLM generation completed in 0.02s
  Response: Hello.
✓ Llama-3.2-1B fast_inference test PASSED

## Test Complete

The Llama-3.2-1B fast_inference test has completed. The kernel will now shut down to release all GPU memory.

**Next:** Run `02_FastInference_Qwen.ipynb` for Qwen3-4B testing.

In [3]:
# Shutdown kernel to release all GPU memory
import IPython
print("Shutting down kernel to release GPU memory...")
app = IPython.Application.instance()
app.kernel.do_shutdown(restart=False)

Shutting down kernel to release GPU memory...

{'status': 'ok', 'restart': False}